## VERSION 4

In [ ]:
from openai import OpenAI
import json
import random
import time
from datetime import datetime
import statistics

# Setup client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

class AdaptiveLearningSystem:
    def __init__(self):
        self.traits = [
            "vocabulary_expansion_count",
            "curiosity_level", 
            "confidence_level",
            "output_compression_ratio",
            "emotional_consistency_score"
        ]
        
        # Track learner progression over time
        self.learning_history = []
        self.current_session = 1
        self.learning_goal = ""
        
        # Task difficulty adaptation
        self.task_difficulty = "medium"
        
    def generate_adaptive_task(self, goal, current_scores=None, session_num=1):
        """Generate tasks that adapt based on learner's current performance"""
        
        # Base task pools by difficulty
        beginner_tasks = [
            "Write 2-3 sentences about why you want to learn this topic.",
            "Describe one thing you already know about this subject.",
            "What questions do you have about this topic?",
            "Write a short paragraph about how you learn best."
        ]
        
        intermediate_tasks = [
            "Recall the last time you felt stuck while learning something similar. What helped you move forward?",
            "Describe a time you had to explain something complex to someone else. What approach did you use?",
            "Write about a moment when you changed your understanding of something. What triggered the shift?",
            "Compare two different ways you could approach learning this topic."
        ]
        
        advanced_tasks = [
            "Analyze a recent challenge you faced and create a systematic approach to solve similar problems.",
            "Write a detailed reflection on how your learning style has evolved over time.",
            "Construct an argument for why someone should learn this topic, addressing potential counterarguments.",
            "Design a learning plan for someone else who wants to master this topic."
        ]
        
        # Adapt task difficulty based on performance
        if current_scores:
            avg_score = statistics.mean(current_scores.values())
            if avg_score < 0.4:
                self.task_difficulty = "beginner"
            elif avg_score > 0.7:
                self.task_difficulty = "advanced"
            else:
                self.task_difficulty = "intermediate"
        
        # Select appropriate task pool
        if self.task_difficulty == "beginner":
            task_pool = beginner_tasks
        elif self.task_difficulty == "advanced":
            task_pool = advanced_tasks
        else:
            task_pool = intermediate_tasks
            
        return random.choice(task_pool)

    def score_response(self, goal, task, response):
        """Mock scoring method (Replace with LLM-based scoring if needed)"""
        try:
            return {
                trait: round(random.uniform(0.3, 0.9), 2)
                for trait in self.traits
            }
        except Exception as e:
            print(f"⚠️ Error in scoring: {e}")
            return {trait: 0.5 for trait in self.traits}

    def generate_adaptive_feedback(self, scores, goal, session_num):
        """Generate personalized feedback based on current performance"""
        
        # Analyze performance patterns
        low_scores = {k: v for k, v in scores.items() if v < 0.4}
        high_scores = {k: v for k, v in scores.items() if v > 0.7}
        
        feedback_prompt = f"""You are a personalized learning coach. A student learning "{goal}" just completed session {session_num}.

Their current performance scores:
{json.dumps(scores, indent=2)}

LOW PERFORMANCE AREAS (need improvement):
{list(low_scores.keys())}

HIGH PERFORMANCE AREAS (strengths):
{list(high_scores.keys())}

Learning History Context:
- Current task difficulty: {self.task_difficulty}
- Session number: {session_num}

Provide:
1. **Current State Analysis**: What do these scores tell us about their learning?
2. **Targeted Feedback**: Specific, actionable advice for the lowest 1-2 areas
3. **Next Steps**: What should they focus on in the next session?
4. **Encouragement**: Acknowledge their strengths and progress

Keep response concise but supportive (max 200 words)."""

        try:
            completion = client.chat.completions.create(
                model="deepseek/deepseek-r1-0528-qwen3-8b:free",
                messages=[
                    {"role": "system", "content": "You are an encouraging, insightful learning coach."},
                    {"role": "user", "content": feedback_prompt}
                ],
                temperature=0.6,
                max_tokens=300
            )
            
            return completion.choices[0].message.content.strip()
            
        except Exception as e:
            return f"Unable to generate feedback right now. Focus on improving your lowest scoring areas: {list(low_scores.keys())}"
    
    def track_progress(self, scores, task, response):
        """Track learner progress over time"""
        session_data = {
            "session": self.current_session,
            "timestamp": datetime.now().isoformat(),
            "task": task,
            "response": response,
            "scores": scores,
            "difficulty": self.task_difficulty,
            "avg_score": statistics.mean(scores.values())
        }
        
        self.learning_history.append(session_data)
        
        # Show progress trends
        if len(self.learning_history) > 1:
            prev_avg = self.learning_history[-2]["avg_score"]
            current_avg = session_data["avg_score"]
            trend = "📈 Improving" if current_avg > prev_avg else "📉 Declining" if current_avg < prev_avg else "➡️ Steady"
            print(f"\n📊 PROGRESS TREND: {trend}")
            print(f"Previous avg: {prev_avg:.2f} → Current avg: {current_avg:.2f}")
    
    def run_learning_cycle(self):
        """Main learning loop - the core of the adaptive system"""
        
        print("🎯 ADAPTIVE LEARNING SYSTEM INITIALIZED")
        print("=" * 50)
        
        # Initial setup
        if not self.learning_goal:
            self.learning_goal = input("What are you trying to learn today? ")
        
        print(f"\n🚀 Starting adaptive learning cycle for: {self.learning_goal}")
        print("Type 'quit' to exit at any time.\n")
        
        while True:
            print(f"\n{'='*20} SESSION {self.current_session} {'='*20}")
            
            # Step 1: Generate adaptive task
            previous_scores = self.learning_history[-1]["scores"] if self.learning_history else None
            task = self.generate_adaptive_task(self.learning_goal, previous_scores, self.current_session)
            
            print(f"📝 TASK (Difficulty: {self.task_difficulty.upper()}):")
            print(f"{task}")
            
            # Step 2: Get user response
            print(f"\n✍️ Your response:")
            user_response = input()
            
            if user_response.lower() == 'quit':
                print("📊 Final learning summary:")
                self.show_progress_summary()
                break
                
            # Step 3: Score response
            print("\n🔄 Analyzing your response...")
            scores = self.score_response(self.learning_goal, task, user_response)
            
            # Step 4: Generate adaptive feedback
            feedback = self.generate_adaptive_feedback(scores, self.learning_goal, self.current_session)
            
            # Step 5: Track progress
            self.track_progress(scores, task, user_response)
            
            # Step 6: Display results
            print("\n📊 CURRENT METRICS:")
            for trait, value in scores.items():
                status = "🟢" if value > 0.7 else "🟡" if value > 0.4 else "🔴"
                print(f"{status} {trait}: {value:.2f}")
            
            print(f"\n🤖 ADAPTIVE FEEDBACK:")
            print(feedback)
            
            # Step 7: Prepare for next iteration
            self.current_session += 1
            
            print(f"\n⏱️  Press Enter to continue to session {self.current_session} (or type 'quit' to exit)")
            continue_input = input()
            if continue_input.lower() == 'quit':
                print("📊 Final learning summary:")
                self.show_progress_summary()
                break
    
    def show_progress_summary(self):
        """Display overall learning progress"""
        if not self.learning_history:
            print("No learning data to display.")
            return
            
        print(f"\n🎯 LEARNING SUMMARY for: {self.learning_goal}")
        print("=" * 50)
        
        # Calculate progress trends
        first_session = self.learning_history[0]
        last_session = self.learning_history[-1]
        
        print(f"📈 Sessions completed: {len(self.learning_history)}")
        print(f"📊 Overall improvement: {last_session['avg_score'] - first_session['avg_score']:.2f}")
        
        # Show trait improvements
        print("\n📋 TRAIT IMPROVEMENTS:")
        for trait in self.traits:
            first_score = first_session['scores'][trait]
            last_score = last_session['scores'][trait]
            improvement = last_score - first_score
            trend = "📈" if improvement > 0 else "📉" if improvement < 0 else "➡️"
            print(f"{trend} {trait}: {first_score:.2f} → {last_score:.2f} ({improvement:+.2f})")

# Run the system
if __name__ == "__main__":
    system = AdaptiveLearningSystem()
    system.run_learning_cycle()
